## Pima Indian Woman Diabetes

Tenemos un modelo que tiene la informacion de las mujeres indias de Pima, con esta informacion y una prueba de diabetes podemos entrenar nuestro modelo para que prediga si la mujer tiene o no diabetes.

In [4]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('rfcExample').getOrCreate()

21/12/09 19:54:15 WARN Utils: Your hostname, flores-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
21/12/09 19:54:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/12/09 19:54:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
data = spark.read.csv('pima-indians-diabetes.data.csv', inferSchema=True, header=True).toDF("embarazo_meses","concentracion_glucosa","presion_arterial","espesor_triceps","insulina","masa_corporal","pedigri_diabetes","edad","diabetes")

In [4]:
data.printSchema()

root
 |-- embarazo_meses: integer (nullable = true)
 |-- concentracion_glucosa: integer (nullable = true)
 |-- presion_arterial: integer (nullable = true)
 |-- espesor_triceps: integer (nullable = true)
 |-- insulina: integer (nullable = true)
 |-- masa_corporal: double (nullable = true)
 |-- pedigri_diabetes: double (nullable = true)
 |-- edad: integer (nullable = true)
 |-- diabetes: integer (nullable = true)



In [1]:
# Importamos nuestro Vector y Evaluador de precision del modelo
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [6]:
assembler = VectorAssembler(inputCols=["embarazo_meses",
                                       "concentracion_glucosa",
                                       "presion_arterial",
                                       "espesor_triceps",
                                       "insulina",
                                       "masa_corporal",
                                       "pedigri_diabetes",
                                       "edad"],
                           outputCol='features')

In [7]:
output = assembler.transform(data)
output = output.select(['features','diabetes'])

In [8]:
final_data = output.select(['features','diabetes'])

In [72]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [73]:
from pyspark.ml.classification import RandomForestClassifier

In [74]:
rf = RandomForestClassifier(labelCol="diabetes", featuresCol="features", numTrees=30)

## Numero de arboles

El numero de arboles es determinado por la cantidad de datos a utilizar.
La cantidad normal a utilizar es de 10 arboles, 30 arboles y 100 arboles.
Tambien se encontro que cuando se tiene 1 lakh o mas, de datos, se maneja de 200 a 300 arboles.

In [75]:
model = rf.fit(train_data)

In [76]:
predictions = model.transform(test_data)
predictions.select("diabetes","prediction").show(3)

+--------+----------+
|diabetes|prediction|
+--------+----------+
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 3 rows



In [77]:
evaluator = MulticlassClassificationEvaluator(labelCol='diabetes', predictionCol='prediction',metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

## Evaluamos
Se evalua la precision de nuestro modelo

In [78]:
print("Test Error = {}%".format(1.0-accuracy))
print("Accuracy: {}%".format(accuracy))

Test Error = 0.19841269841269837%
Accuracy: 0.8015873015873016%
